在前面的学习中，我们其实已经接触到了很多 Python 对象比较和复制的例子，比如下面这个，判断 a 和 b 是否相等的 if 语句： 
```python
if a == b: 
    ...
```

再比如第二个例子，这里 l_2 就是 l_1 的拷贝。
```python
l_1 = [1,2,3]
l_2 = list(l_1)
```

但你可能并不清楚，这些语句的背后发生了什么。比如， 
1. l2 是 l1 的浅拷贝（shallow copy）还是深度拷贝（deep copy）呢？ 
2. a == b 是比较两个对象的值相等，还是两个对象完全相等呢？ 

关于这些的种种知识，我希望通过这节课的学习，让你有个全面的了解。

# '==' vs 'is'

等于（==）和 is 是 Python 中对象比较常用的两种方式。

'=='操作符比较对象之间的值是否相等，比如下面的例子，表示比较变量 a 和 b 所指向的值是否相等。

'is'操作符比较的是对象的身份标识是否相等，即它们是否是同一个对象，是否指向同一个内存地址。

In [12]:
a = 10
b = 10
a == b

True

在 Python 中，每个对象的身份标识，都能通过函数 `id(object)` 获得。因此，'is'操作符，相当于比较对象之间的 ID 是否相等

In [13]:
print(id(a), id(b))

140722626880584 140722626880584


In [14]:
a is b

True

首先 Python 会为 10 这个值开辟一块内存，然后变量 a 和 b 同时指向这块内存区域，即 a 和 b 都是指向 10 这个变量，因此 a 和 b 的值相等，id 也相等，a == b和a is b都返回 True。

不过，需要注意，对于整型数字来说，以上a is b为 True 的结论，**只适用于 -5 到 256 范围内的数字**。 Python 内部会对 -5 到 256 的整型维持一个数组，起到一个缓存的作用。这样，每次你试图创建一个 -5 到 256 范围内的整型数字时，Python 都会从这个数组中返回相对应的引用，而不是重新开辟一块新的内存空间。如果整型数字超过了这个范围，比如上述例子中的 257，Python 则会为两个 257 开辟两块内存区域，因此 a 和 b 的 ID 不一样，a is b就会返回 False 了。

关于python的存储问题

1. 由于python中万物皆对象，所以python的存储问题是对象的存储问题，并且对于每个对象，python会分配一块内存空间去存储它
2. 对于整数和短小的字符等，python会执行缓存机制，即将这些对象进行缓存，不会为相同的对象分配多个内存空间
3. 容器对象，如列表、元组、字典等，存储的其他对象，仅仅是其他对象的引用，即地址，并不是这些对象本身

比如下面这个例子：

In [11]:
a = 257
b = 257
print(a == b)
print(id(a), id(b))
print(a is b)

True
2348929507888 2348929508112
False


通常来说，在实际工作中，当我们比较变量时，使用'=='的次数会比'is'多得多，因为我们一般更关心两个变量的值，而不是它们内部的存储地址。但是，当我们比较一个变量与一个单例（singleton）时，通常会使用'is'。一个典型的例子，就是检查一个变量是否为 None：
```python
if a is None:
      ...

if a is not None:
      ...

```

这里注意，比较操作符'is'的速度效率，通常要优于'=='。因为'is'操作符不能被重载，这样，Python 就不需要去寻找，程序中是否有其他地方重载了比较操作符，并去调用。执行比较操作符'is'，就仅仅是比较两个变量的 ID 而已。

但是'=='操作符却不同，执行a == b相当于是去执行`a.__eq__(b)`，而 Python 大部分的数据类型都会去重载__eq__这个函数，其内部的处理通常会复杂一些。比如，对于列表，__eq__函数会去遍历列表中的元素，比较它们的顺序和值是否相等。

不过，对于不可变（immutable）的变量，如果我们之前用'=='或者'is'比较过，结果是不是就一直不变了呢？ 答案自然是否定的。我们来看下面一个例子：

In [15]:
t1 = (1, 2, [3, 4])
t2 = (1, 2, [3, 4])
t1 == t2



True

In [16]:

t1[-1].append(5)
t1 == t2

False

我们知道元组是不可变的，但元组可以嵌套，它里面的元素可以是列表类型，列表是可变的，所以如果我们修改了元组中的某个可变元素，那么元组本身也就改变了，之前用'is'或者'=='操作符取得的结果，可能就不适用了。

# 浅拷贝shallow copy和深度拷贝deep copy

**浅拷贝是指重新分配一块内存，创建一个新的对象**，里面的元素时原对象中子对象的引用。如下面的例子，当使用数据类型本身的构造器的时候，会产生浅拷贝。

可以看出，连个list的值是一样的，但是对应的内存位置是不一样的。

In [21]:
l_1 = [1,2,3]
l_2 = list(l_1)
print(l_1, l_2)
print(l_1 == l_2)
print(l_1 is l_2)

[1, 2, 3] [1, 2, 3]
True
False


对于可变长度的对象，还可以通过切片操作符`:`完成浅拷贝.

In [23]:
l_1 = [1,2,3]
l_2 = l_1[:]
print(l_1, l_2)
print(l_1 == l_2)
print(l_1 is l_2)

[1, 2, 3] [1, 2, 3]
True
False


但是也有例外，对于元组来说，如果使用`tuple()`或者切片的操作符`:`,不会创建一份浅拷贝。相反，它会返回一个指向相同元组的引用：

下面的例子可以看出，元组 (1, 2, 3) 只被创建一次，t1 和 t2 同时指向这个元组。

In [24]:
t1 = (1, 2, 3) 
t2 = tuple(t1)

print(t1==t2)
print(t1 is t2)

True
True


**如果原对象中的元素不可变，那倒无所谓；但如果元素可变，浅拷贝通常会带来一些副作用**，尤其需要注意。我们来看下面的例子：

这个例子中，我们首先初始化了一个列表 l1，里面的元素是一个列表和一个元组；然后对 l1 执行浅拷贝，赋予 l2。因为浅拷贝里的元素是对原对象元素的引用，因此 l2 中的元素和 l1 指向同一个列表和元组对象。

接着往下看。`l1.append(100)`，表示**对 l1 的列表新增元素 100**。这个操作不会对 l2 产生任何影响，因为 **l2 和 l1 作为整体是两个不同的对象，并不共享内存地址**。操作过后 l2 不变，l1 会发生改变。

再来看，`l1[0].append(3)`，这里表示对 l1 中的第一个列表新增元素 3。**因为 l2 是 l1 的浅拷贝，l2 中的第一个元素和 l1 中的第一个元素，共同指向同一个列表，因此 l2 中的第一个列表也会相对应的新增元素 3**。操作后 l1 和 l2 都会改变。

最后是`l1[1] += (50, 60)`，因为**元组是不可变的，这里表示对 l1 中的第二个元组拼接，然后重新创建了一个新元组作为 l1 中的第二个元素，而 l2 中没有引用新元组，因此 l2 并不受影响**。操作后 l2 不变，l1 发生改变。

In [30]:
l1 = [[1, 2], (30, 40)] 
l2 = list(l1) 
l1.append(100) 
l1[0].append(3)

print(f'l1 is :   {l1}')
print(f'l2 is :   {l2}')



l1[1] += (50,60)
print(f'l1 is ::   {l1}')
print(f'l2 is ::   {l2}')


l1 is :   [[1, 2, 3], (30, 40), 100]
l2 is :   [[1, 2, 3], (30, 40)]
l1 is ::   [[1, 2, 3], (30, 40, 50, 60), 100]
l2 is ::   [[1, 2, 3], (30, 40)]


深度拷贝，是指重新分配一块内存，创建一个新的对象，并且将原对象中的元素，以递归的方式，通过创建新的子对象拷贝到新对象中。因此，新对象和原对象没有任何关联。

Python 中以 `copy.deepcopy()` 来实现对象的深度拷贝。比如上述例子写成下面的形式，就是深度拷贝。

无论 l1 如何变化，l2 都不变。因为此时的 l1 和 l2 完全独立，没有任何联系。

In [31]:
import copy

l1 = [[1, 2], (30, 40)] 
l2 = copy.deepcopy(l1) 
l1.append(100) 
l1[0].append(3)

print(l1)
print(l2)

[[1, 2, 3], (30, 40), 100]
[[1, 2], (30, 40)]


深度拷贝也不是完美的，往往也会带来一系列问题。如果被拷贝对象中存在指向自身的引用，那么程序很容易陷入无限循环：

In [33]:
import copy
x = [1]
x.append(x)

print(x)


y = copy.deepcopy(x)
print(y)


[1, [...]]
[1, [...]]


列表 x 中有指向自身的引用，因此 x 是一个无限嵌套的列表。但是我们发现深度拷贝 x 到 y 后，程序并没有出现 stack overflow 的现象。

这是因为深度拷贝函数 deepcopy 中会维护一个字典，记录已经拷贝的对象与其 ID。拷贝过程中，如果字典里已经存储了将要拷贝的对象，则会从字典直接返回，我们来看相对应的源码就能明白

In [ ]:
def deepcopy(x, memo=None, _nil=[]):
    """Deep copy operation on arbitrary Python objects.
    See the module's __doc__ string for more info.
    """
    
    if memo is None:
        memo = {}
    d = id(x) # 查询被拷贝对象x的id
    y = memo.get(d, _nil) # 查询字典里是否已经存储了该对象
    if y is not _nil:
        return y # 如果字典里已经存储了将要拷贝的对象，则直接返回
        ...    


# 总结 

1. 比较操作符'=='表示比较对象间的值是否相等，而'is'表示比较对象的标识是否相等，即它们是否指向同一个内存地址。 
2. 比较操作符'is'效率优于'=='，因为'is'操作符无法被重载，执行'is'操作只是简单的获取对象的 ID，并进行比较；而'=='操作符则会递归地遍历对象的所有值，并逐一比较。 
3. 浅拷贝中的元素，是原对象中子对象的引用，因此，如果原对象中的元素是可变的，改变其也会影响拷贝后的对象，存在一定的副作用。 
4. 深度拷贝则会递归地拷贝原对象中的每一个子对象，因此拷贝后的对象和原对象互不相关。另外，深度拷贝中会维护一个字典，记录已经拷贝的对象及其 ID，来提高效率并防止无限递归的发生。

下面的例子看是重新定义了一个l_2,但其实不是拷贝的问题，而且l_2指向l_1,共用一个内存地址，不同的别名。

In [35]:
l_1 = [1,2,3]
l_2 = l_1
print(l_1, l_2)
print(l_1 == l_2)
print(l_1 is l_2)

l_1[1] += 2
print(l_1, l_2)
print(l_1 == l_2)
print(l_1 is l_2)

l_2[1] += 2
print(l_1, l_2)
print(l_1 == l_2)
print(l_1 is l_2)


[1, 2, 3] [1, 2, 3]
True
True
[1, 4, 3] [1, 4, 3]
True
True
[1, 6, 3] [1, 6, 3]
True
True


| 类型 | 是否创建新的对象 | 内存地址 | 初始内容 | 内容传递方式 | 可变对象操作 | 
| ------- |  -------------- | ------- | ---- | ---------- | ----------- |
| 浅拷贝  | 是 | 不同 | 相同 | 对原对象中的子对象的引用 | 同步变化 |
| 深拷贝  | 是 | 不同 | 相同 | 递归的方式拷贝到新对象中 | 不同步变化 |
| 引用 | 否 | 相同 | 相同 | 指向同一内存地址 | 同步变化|


# 思考题 

深度拷贝，拷贝一个无限嵌套的列表。那么。当我们用等于操作符'=='进行比较时，输出会是什么呢？是 True 或者 False 还是其他？为什么呢？

In [36]:
import copy
x = [1]
x.append(x)

y = copy.deepcopy(x)

# 以下命令的输出是？
x == y


RecursionError: maximum recursion depth exceeded in comparison

程序会报错，因为x是一个无限嵌套的列表，y深拷贝于x，按道理来讲 x == y应该是True的，但进行比较操作符“==”的时候，'=='操作符则会递归地遍历对象的所有值，并逐一比较。而python为了防止栈崩溃，递归的层数是要限定的，不会无休下去，所以到了限定的层数，python解释器会跳出错误。执行了一下代码，也的确是跳出了 RecursionError: maximum recursion depth exceeded in comparison。